In [ ]:
!pip install gcloud

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.4/454.4 kB 10.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for gcloud: filename=gcloud-0.18.3-py3-none-any.whl size=602927 sha256=1bc53ab1e465f0801836f2fda23e48775a47b0e76695a269b7486cf8115c1d24
  Stored in directory: /root/.cache/pip/wheels/7c/30/88/5017af921da3a33af785f0d0fd3e944b845bc62a445a2c2f69
Successfully built gcloud


In [ ]:
!gcloud auth application-default login

Go to the following link in your browser, and complete the sign-in prompts:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=https%3A%2F%2Fsdk.cloud.google.com%2Fapplicationdefaultauthcode.html&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login&state=0ogYZdcu3p86IwREgToPLJoi79IXQ1&prompt=consent&token_usage=remote&access_type=offline&code_challenge=-91xJVRYht3812sG9PODxnGqsSaOGUnQtvF1FDwOJZE&code_challenge_method=S256

Once finished, enter the verification code provided in your browser: 4/0AcvDMrCAS94n9mTFVb3EiPxjxPlYIrjAlgYiGLvVfD-I4lAqrftJf0wKYKRWXo70-5W2sw

Credentials saved to file: [/content/.config/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).
Ca

In [ ]:
##importando pacotes
import pandas as pd
import numpy as np
import time
import os
import pandas_gbq
from google.cloud import bigquery
import glob
import openpyxl
import csv
import re

In [ ]:
client = bigquery.Client(project='repositoriodedadosgpsp')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [ ]:
# Definir a consulta SQL
query = """
SELECT ano, 'publico' as flag_publico_privado,variavel,categoria, media_remuneracao FROM `repositoriodedadosgpsp.Datalake.RAIS_remuneracao_vinculos_publicos_v2` union all SELECT ano, flag_publico_privado,variavel, categoria,media_remuneracao FROM `repositoriodedadosgpsp.Datalake.RAIS_remuneracao_vinculos_privados_v2`
"""



# Executar a consulta e salvar os resultados em um DataFrame do pandas
df = client.query(query).to_dataframe()

# Exibir os primeiros registros do DataFrame
print(df.head())

    ano flag_publico_privado variavel         categoria  media_remuneracao
0  2022              publico   genero  Não identificado           0.000000
1  2022              publico   genero          Feminino        5099.993482
2  2017              publico   genero          Feminino        3791.736833
3  2009              publico   genero          Feminino        1887.628480
4  2008              publico   genero          Feminino        1723.506923


In [ ]:
#query = "SELECT ano, 'publico' as flag_publico_privado,variavel,categoria, media_remuneracao FROM `repositoriodedadosgpsp.Datalake.RAIS_remuneracao_vinculos_publicos` union all SELECT ano, flag_publico_privado,variavel, categoria,media_remuneracao FROM `repositoriodedadosgpsp.Datalake.RAIS_remuneracao_vinculos_privados`"

In [ ]:
df

,ano,flag_publico_privado,variavel,categoria,media_remuneracao
0,2022,publico,genero,Não identificado,0.000000
1,2022,publico,genero,Feminino,5099.993482
2,2017,publico,genero,Feminino,3791.736833
3,2009,publico,genero,Feminino,1887.628480
4,2008,publico,genero,Feminino,1723.506923
...,...,...,...,...,...
1019,2015,privado,poderes,Outros,2014.787251
1020,2009,privado,nivel_governo,Outros,1203.932601
1021,2009,privado,poderes,Outros,1203.932601
1022,2018,privado,poderes,Outros,2340.963919


In [ ]:
## O privado vem com a série inteira, vou consertar essa parte ao consertar a série

df = df[df['ano']>2002]


In [ ]:
df=df[df['variavel']!='tipo_vinculo']

In [ ]:
df['ano'].unique()

<IntegerArray>
[2022, 2017, 2009, 2008, 2013, 2019, 2007, 2010, 2018, 2016, 2020, 2011, 2015,
 2003, 2006, 2005, 2012, 2014, 2004, 2021]
Length: 20, dtype: Int64

In [19]:
# Definir a consulta SQL
query = """
SELECT * FROM `repositoriodedadosgpsp.Datalake.ipca`
"""



# Executar a consulta e salvar os resultados em um DataFrame do pandas
ipca = client.query(query).to_dataframe()

# Exibir os primeiros registros do DataFrame
print(ipca.head())

    ano  acumulado_dezembro_ano           it  fator_correcao
0  2018                    3.75  4098.727240        1.328049
1  2011                    6.50  2734.909768        1.990308
2  2020                    4.52  4468.629667        1.218117
3  2006                    3.14  2101.282770        2.590471
4  1996                    9.56  1095.600000        4.968339


In [20]:
ipca.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29 entries, 0 to 28
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   ano                     29 non-null     Int64  
 1   acumulado_dezembro_ano  29 non-null     float64
 2   it                      29 non-null     float64
 3   fator_correcao          29 non-null     float64
dtypes: Int64(1), float64(3)
memory usage: 1.1 KB


In [23]:
df1= df.merge(ipca[['ano','fator_correcao']], right_on='ano',left_on='ano')

In [24]:
df1

,ano,flag_publico_privado,variavel,categoria,media_remuneracao,fator_correcao
0,2022,publico,genero,Não identificado,0.000000,1.046200
1,2022,publico,genero,Feminino,5099.993482,1.046200
2,2022,publico,genero,Masculino,7201.277814,1.046200
3,2022,publico,poderes,Outros,8540.517166,1.046200
4,2022,publico,poderes,Executivo,4486.838805,1.046200
...,...,...,...,...,...,...
467,2021,privado,grau_instrucao,Até Fundamental,1786.915974,1.106775
468,2021,privado,grau_instrucao,Até Ensino Médio,2065.832710,1.106775
469,2021,privado,grau_instrucao,Até Ensino Superior Completo,5586.401470,1.106775
470,2021,privado,poderes,Outros,2555.798369,1.106775


In [26]:
df1['media_remuneracao_ajustada']= df1['media_remuneracao']* df1['fator_correcao']

In [27]:
df1

,ano,flag_publico_privado,variavel,categoria,media_remuneracao,fator_correcao,media_remuneracao_ajustada
0,2022,publico,genero,Não identificado,0.000000,1.046200,0.000000
1,2022,publico,genero,Feminino,5099.993482,1.046200,5335.613181
2,2022,publico,genero,Masculino,7201.277814,1.046200,7533.976849
3,2022,publico,poderes,Outros,8540.517166,1.046200,8935.089059
4,2022,publico,poderes,Executivo,4486.838805,1.046200,4694.130758
...,...,...,...,...,...,...,...
467,2021,privado,grau_instrucao,Até Fundamental,1786.915974,1.106775,1977.713891
468,2021,privado,grau_instrucao,Até Ensino Médio,2065.832710,1.106775,2286.411957
469,2021,privado,grau_instrucao,Até Ensino Superior Completo,5586.401470,1.106775,6182.889375
470,2021,privado,poderes,Outros,2555.798369,1.106775,2828.693689


In [28]:
df1.columns

Index(['ano', 'flag_publico_privado', 'variavel', 'categoria',
       'media_remuneracao', 'fator_correcao', 'media_remuneracao_ajustada'],
      dtype='object')

In [29]:
df1 = df1[['ano','flag_publico_privado', 'variavel', 'categoria', 'media_remuneracao',
       'fator_correcao', 'media_remuneracao_ajustada']]

In [30]:
df1['variavel'].unique()

array(['genero', 'poderes', 'nivel_governo', 'grau_instrucao'],
      dtype=object)

In [31]:
df1['flag_publico_privado']=np.where(df1['flag_publico_privado']=="privado","Privado",df1['flag_publico_privado'])
df1['flag_publico_privado']=np.where(df1['flag_publico_privado']=="publico","Público",df1['flag_publico_privado'])
df1

,ano,flag_publico_privado,variavel,categoria,media_remuneracao,fator_correcao,media_remuneracao_ajustada
0,2022,Público,genero,Não identificado,0.000000,1.046200,0.000000
1,2022,Público,genero,Feminino,5099.993482,1.046200,5335.613181
2,2022,Público,genero,Masculino,7201.277814,1.046200,7533.976849
3,2022,Público,poderes,Outros,8540.517166,1.046200,8935.089059
4,2022,Público,poderes,Executivo,4486.838805,1.046200,4694.130758
...,...,...,...,...,...,...,...
467,2021,Privado,grau_instrucao,Até Fundamental,1786.915974,1.106775,1977.713891
468,2021,Privado,grau_instrucao,Até Ensino Médio,2065.832710,1.106775,2286.411957
469,2021,Privado,grau_instrucao,Até Ensino Superior Completo,5586.401470,1.106775,6182.889375
470,2021,Privado,poderes,Outros,2555.798369,1.106775,2828.693689


In [32]:
schema=[bigquery.SchemaField('ano','INTEGER',description='Ano de referência'),
bigquery.SchemaField('flag_publico_privado','STRING',description='Se se refere ao setor publico ou privado'),
 bigquery.SchemaField('variavel','STRING',description='De qual variável aquela remuneração se refere'),
 bigquery.SchemaField('categoria','STRING',description='Categoria dentro daquela variável'),
 bigquery.SchemaField('media_remuneracao','FLOAT',description='Valor nominal da média de remuneração daquele ano'),
bigquery.SchemaField('fator_correcao','FLOAT',description='índice para correção monetária baseando-se no IPCA para dezembro de 2022'),
bigquery.SchemaField('media_remuneracao_ajustada','FLOAT',description='Valor nominal da média de remuneração daquele ano ajustado para dezembro de 2022')
 ]


In [33]:
client = bigquery.Client(project='repositoriodedadosgpsp')

/usr/local/lib/python3.10/dist-packages/google/auth/_default.py:76: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [34]:
dataset_ref = client.dataset('perfil_remuneracao')

In [35]:
table_ref = dataset_ref.table('RAIS_remuneracao_vinculos_publicos_privados_v_3')
job_config = bigquery.LoadJobConfig(schema=schema)
job = client.load_table_from_dataframe(df1, table_ref, job_config=job_config)
job.result()

LoadJob<project=repositoriodedadosgpsp, location=US, id=fc2fa796-8621-4346-8c3f-99ce2f8d7b9a>

In [ ]:
#para excluir tabelas
#client.delete_table('perfil_remuneracao.RAIS_remuneracao_vinculos_publicos_privados', not_found_ok=True)